# Data Collection Subject

How does the economy impact the popularity of movie generes?

# Scraping Data on Movies

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
url = 'https://www.the-numbers.com/market/genres'
html = requests.get(url).content
soup = BeautifulSoup(html,'lxml')

In [4]:
data = soup.find_all('table')[0].get_text()
newdata = data.split('\n')

for count in range(newdata.count('')):
    newdata.remove('')
    
newdata = newdata[1:]

genres=[]
for index in range(len(newdata)):
    if (index % 6)==1:
        item = newdata[index]
        genres.append(item)

In [5]:
newgenre=genres[:8]
for i in range(len(newgenre)):
    if newgenre[i] == 'Thriller/Suspense': 
        newgenre[i]= 'Thriller-or-Suspense'
    
    if newgenre[i] == 'Romantic Comedy':
        newgenre[i] = 'Romantic-Comedy'
    
    if newgenre[i] == 'Black Comedy':
        newgenre[i] = 'Black-Comedy'

In [6]:
#two dimensional list where dl[index] contains all of the market share datapoints by year for a genre
dl =[]
for genre in newgenre:
    #market share for each genre
    datalist=[]
    url = 'https://www.the-numbers.com/graphs/market/selected-item/genres/genre/gengres/'+genre+'/'+genre+'#_ABSTRACT_RENDERED_ID_0'
    html = requests.get(url).content
    soup = BeautifulSoup(html,'lxml')
    soup.find_all('script')[1].get_text()
    
    text =soup.find_all('script')[1].get_text()
    newtext = text.split(';')
    twotext = newtext[2].split('\t')

    for count in range(twotext.count('')):
        twotext.remove('')

    threetext = twotext[3][1:].split(', ')


    for index in range(len(threetext)):
        data = threetext[index].split(',')
        year=data[0][2:6]
        datapoint=float(data[2][:4])
        datalist.append(datapoint/100)
    
    dl.append(datalist)

#number of movies produced
dlnummade=[]
for genre in newgenre:
    url='https://www.the-numbers.com/market/genre/'+genre
    html = requests.get(url).content
    soup = BeautifulSoup(html,'lxml')
    p2text = soup.find_all('table')[1].get_text()
    makesomepoints= p2text.split('\n')

    for x in range(makesomepoints.count('')):
        makesomepoints.remove('')

    tabledata = makesomepoints[1:]
    pointsingenre=[]

    for s in range(len(tabledata)):
        if s%8==1:
            pointsingenre.append(tabledata[s])
    dlnummade.append(pointsingenre)

In [39]:
newdata=pd.DataFrame()
#append market share
index = 0
for thegenre in newgenre:
    if len(dl[index])!=26:
        dl[index].insert(0,0)
    newdata[thegenre + ' Market Share']=dl[index]
    index += 1
if len(newdata)!=24:
    newdata = newdata.iloc[:-2]

other=[]
for x in list(range(1995,2019)):
    other.append(1-final_data.loc[x][:8].sum())
newdata["Other Genres Market Share"] = other
    
gdp = pd.read_csv("gdp.csv")[(1995-1961):]
gdp.reset_index(inplace=True)
newdata['GDP % Growth']= gdp[' GDP Growth (%)']
newdata['GDP Annual Change']= gdp[' Annual Change']


pce = pd.read_csv("pce.csv")
pcedata=[]
for index in range(len(pce.loc[0])):
    pcedata.append(pce.iloc[0][index])
newdata['PCE'] = pcedata[2:]

final_data = newdata.rename(index = lambda s: s + 1995)

In [45]:
final_data.to_csv('data')

SyntaxError: invalid syntax (<ipython-input-45-b43ee6f8a459>, line 1)